In [1]:
# calculate daily anomalies in a 1000km radius around the storm centers identified with Tempest Extremes
# store the results as netcdf files
#
# STEPS:
#  for each day, find relevant storm centers
#  get the area within 1000km around the storm center
#  a) at each location within this area, subtract the daily climatology 1997-2018 (non-seasonal variability is left)
#  b) at each location within this area, subtract the average over the 5 preceeding days ("storm-induced anomalies")
#
# Notes: 
#  sea-ice area is ignored (all grid cells with sea-ice concentration >0 are masked)
#  if _plus_XX_days is processed, the same 5 days are used for averaging as when the original storm positions/timings are used as input
#
# original script name: save_CESM_daily_chl_anomalies_v12_with_HR_and_emulator.ipynb

In [2]:
import os
from tqdm import tqdm
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from netCDF4 import Dataset, MFDataset
import numba as nb
import time as timing
from numba import njit 
from math import sin, cos, sqrt, atan2, radians
from geopy.distance import distance
import seawater as sw

In [3]:
#----
# load storm tracks from tempest extremes
#----

pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'

time_string   = '1997_2018'  # '1979_2018' '2009_2018', 1979_1988, 2009_2015
region_string = '4080S'

#storm_string = 'all_at_noon'
storm_string = 'all_at_noon_plus_4_days'

if len(storm_string)>12: # shifted time
    if storm_string[12]=='p':
        days_shift = int(storm_string[-6:-5]) # make sure to use the exact same 5 days in the calculation as for "day of passing"
    else: 
        days_shift = 0 # for "minus XX days", use the 5 preceeding days from that time
else:
    days_shift = 0 # no shift
print('Time is shifted by this number of days:',days_shift)

fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_'+storm_string[4:]+'.txt'

#---
# read all storm positions for year1
#---

year_storm  = []
month_storm = []
day_storm   = []
hour_storm  = []
index_lon   = []
index_lat   = []
lon_storm   = []
lat_storm   = []
index_storm = []
slp_center  = []
wind_max    = []
count_storms = []
duration_storm = []
counter = 0
with open(pathTE+fileTE, 'r') as f:
    for line in tqdm(f):
        aux = line.split()
        if aux[0]!='start': #year1: # starting a new time entry
            # 262     77      147.375000      -46.333780      1.003353e+05    1.493876e+01    2012    1       2       21
            index_lon   += [int(aux[0])]
            index_lat   += [int(aux[1])]
            if float(aux[2])>180: # convert from 0:360 to -180:180
                aux1 = float(aux[2])-360
            else:
                aux1 = float(aux[2])
            index_storm += [int(aux[10])]
            lon_storm   += [aux1] #[float(aux[2])]
            lat_storm   += [float(aux[3])]
            year_storm  += [int(aux[6])]
            month_storm += [int(aux[7])]
            day_storm   += [int(aux[8])]
            hour_storm  += [int(aux[9])]
            slp_center  += [float(aux[4])]
            wind_max    += [float(aux[5])]
            count_storms+= [counter]
        else:
            # start   29      2012    1       15      21
            duration_storm+= [int(aux[1])]
            counter=counter+1
        del aux
        
# convert from 0:360 to -180:180
#lon_storm[lon_storm>180] = lon_storm[lon_storm>180]-360
print('min/max lon:',np.min(lon_storm),np.max(lon_storm))

print(len(lon_storm),len(year_storm),len(month_storm),len(slp_center))
print('Number of storms:',np.max(count_storms))
#print('avg. duration of storms:',np.mean(duration_storm)) # -> incorrect becaue I reduce storm track to all time steps at noon

print()
print('min/max year:',np.min(year_storm),np.max(year_storm))
print('min/max month:',np.min(month_storm),np.max(month_storm))
print('min/max day:',np.min(day_storm),np.max(day_storm))
print('min/max hour:',np.min(hour_storm),np.max(hour_storm))

print ('done')


4


53876it [00:00, 360010.26it/s]

min/max lon: -179.4375 180.0
44258 44258 44258 44258
Number of storms: 9618

min/max year: 1997 2019
min/max month: 1 12
min/max day: 1 31
min/max hour: 12 12
done


In [5]:
#---
# get lat/lon
#---

path1 = '/global/cfs/cdirs/m4003/cnissen/CESM_chl/'
file_chl = 'totChl_JRA_grid_1980-01-01.nc'

ff2  = xr.open_dataset(path1+file_chl)
lat  = ff2['latitude'].values 
lon  = ff2['longitude'].values
time = ff2['time'].values
ff2.close()

# convert from 0:360 to -180:180
lon[lon>180] = lon[lon>180]-360

print(lat.shape,lon.shape,time.shape)
print(np.min(lon),np.max(lon))
print(np.min(lat),np.max(lat))
print(np.min(time),np.max(time)) 
# -> time in file suggests each day is centred around midnight, but I suspect it should be centered around noon instead!
# (note that I added this to the file!)


(320,) (640,) (365,)
-179.4375 180.0
-89.57009 89.57009
1980-01-01 00:00:00 1980-12-31 00:00:00


In [6]:
#---
# get area for JRA mesh
#---

# get resolution of JRA data
print(np.unique(np.diff(lon)))
print(np.unique(np.diff(lat)))

# need the box boundaries for area calculation
res = 0.5625
xi = np.arange(0,360+res,res)
yi = np.arange(-90,90+res,res)
print(lon.shape,lat.shape)
print(xi.shape,yi.shape)

area = np.zeros((len(xi)-1,len(yi)-1))
#calculate area
for i in range(0,len(xi)-1): #laenge pruefen!
    if i==np.round(len(xi)/2): 
        print ('50% done')
    for j in range(0,len(yi)-1): 
        dist1 = sw.dist([yi[j],yi[j+1]],[xi[i],xi[i]])
        dist2 = sw.dist([yi[j],yi[j]],[xi[i],xi[i+1]])
        area[i][j] = float(dist1[0]) * float(dist2[0]) *1000 *1000 #m2
area = area.transpose()
print ('Global area:',np.sum(area),'m2') 
print(area.shape)


[-359.4375    0.5625]
[0.5569153  0.56020355 0.5609436  0.5612259  0.5613632  0.5614395
 0.5614929  0.5615158  0.5615387  0.56155396 0.5615616  0.56157684
 0.5615845  0.5615921  0.56159973 0.56160736 0.561615   0.5616188
 0.5616207  0.56162167 0.5616217  0.5616218  0.5616219  0.56162214
 0.5616226 ]
(640,) (320,)
(641,) (321,)
50% done
Global area: 509373217207890.1 m2
(320, 640)


In [7]:
#---
# FUNCTIONS
#---

@njit
def get_closest_grid_point(lon_point, lat_point, lon2, lat2):  
    # in all nodes in mesh, return the index of the closest node to lon_point/lat_point
    # lon2 & lat2 are the locations in the new mesh (to be redistributed to)
    # lon2 & lat2 should be in radians
    # numpy needs to be imported outside the function
    
    #from math import sin, cos, sqrt, atan2, radians
    #import numpy as np
    # approximate radius of earth in km
    R = 6371.0
    
    #lat2 = radians(mesh.y2) # all positions in mesh
    #lon2 = radians(mesh.x2)
    #lat2 = [radians(x) for x in mesh.y2]
    #lon2 = [radians(x) for x in mesh.x2]
    lat1 = radians(lat_point)
    lon1 = radians(lon_point)
    bb1 = cos(lat1)
    
    all_distances = np.zeros(len(lon2))
    for i in range(0,len(lon2)):
        dlon = lon2[i] - lon1
        dlat = lat2[i] - lat1
        a = sin(dlat / 2)**2 + bb1 * cos(lat2[i]) * sin(dlon / 2)**2
        all_distances[i] = 2*R*atan2(sqrt(a), sqrt(1 - a)) 
    index_closest_node = np.argmin(all_distances)
    distance_closest_node = np.min(all_distances)

    return index_closest_node, distance_closest_node

def mask_chl_field(points_filtered,lat,lon,data_mask):
        # project all points within XX km of storm center back onto JRA mesh 
        # -> create mask to be used for averaging input fields, e.g., chl
        for kk in range(0,len(points_filtered)):
            index1 = np.where(lon==points_filtered[kk][1])[0]
            index2 = np.where(lat==points_filtered[kk][0])[0]
            data_mask[index2,index1] = 1
        return data_mask
    

In [9]:
#---
# OPTIONAL: store "time" in original daily files
#---
# in regridding, the time variable was not correctly stored
# for each variable, the below code had to be run once so that the time variable is correct in file

add_time_to_file = False

if add_time_to_file:
    save_netcdf = True
    fv = -999

    vari = 'MLD' # FG_CO2_2 or totChl or ECOSYS_IFRAC_2

    if vari in ['totChl']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_chl/'
    elif vari in ['FG_CO2_2']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_fco2/'
    elif vari in ['ECOSYS_IFRAC_2']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_sic/'
    elif vari in ['MLD']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_MLD_regridded/'
    elif vari in ['SST']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_SST_regridded/'
    elif vari in ['wind_speed']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/JRA_wind_speed/'
    elif vari in ['photoC_total_surf']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_total_srf_photoC_regridded/'
    elif vari in ['cloudfrac_isccp']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/CESM_cloudfrac_isccp_regridded/'
    elif vari in ['PAR_incoming']:
        path_daily_mean   = '/global/cfs/cdirs/m4003/cnissen/JRA_PAR_incoming/'

    year_list2 = np.arange(1979,2018+1) #9+1,1)
    time_all   = np.arange(0,365*len(year_list2)+1,1)
    
    
    #year_list2 = np.arange(1978,1978+1,1)#1979,2018+1) #9+1,1)
    #time_all   = np.arange(-365,-1+1,1) #np.arange(0,365*len(year_list2)+1,1)

    for yy in tqdm(range(17,len(year_list2))):

     #   # daily data
     #   file1 = 'totChl_JRA_grid_'+str(year_list2[yy])+'-01-01.nc'
     #   ff = xr.open_dataset(path_daily_mean+file1)
     #   data1 = np.squeeze(ff['totChl']).values
     #   lat = np.squeeze(ff['latitude']).values
     #   lon = np.squeeze(ff['longitude']).values
     #   ff.close()

        # store in new file
        if save_netcdf:
            time = time_all[yy*365:(yy+1)*365]
            netcdf_name = vari+'_JRA_grid_'+str(year_list2[yy])+'-01-01.nc'
            w_nc_fid = Dataset(path_daily_mean+netcdf_name, 'r+', format='NETCDF4_CLASSIC')
            try: # if time variable does not exist, define it first
                w_nc_var1 = w_nc_fid.createVariable('time', 'f4',('time'),fill_value=fv)
                w_nc_var1.units = 'days since '+str(year_list2[0])+'-01-01 00:00:00'
                w_nc_var1.calendar = 'noleap'  
            except: 
                pass
            w_nc_fid.variables['time'][:] = time
            w_nc_fid.close()  


In [10]:
%%time
#---
# LOAD DATA
#---

# define years; important: load one year more here to correcty treat the beginning of the first year of interest!
year_list = np.arange(int(time_string[0:4])-1,int(time_string[5:9])+1,1) 

# optional test plots (These were used in development of script, i.e., for a single day of a single year. If these are set to true, a lot of plots are created...)
test_plot_masked_field  = False
test_plot_combined_mask = False
test_plot_mask          = False

save_netcdf = True

#----
# choose a list of variables to be processed
#----

vari_list = ['totChl','totChl_emulator','totChl_hr']
# ALL VARIABLES: 
# totChl, totChl_hr, totChl_emulator
# PAR_incoming, MLD, MLD_hr, wind_speed, SST, slp, cloudfrac_isccp
# diat_specific_growth_rate_surf, sp_specific_growth_rate_surf, photoC_total_surf, photoC_zint
# sp_Fe_lim_surf, sp_N_lim_surf, sp_P_lim_surf
# diat_Fe_lim_surf, diat_N_lim_surf, diat_P_lim_surf, diat_SiO3_lim_surf

# "storm-induced anomalies" or "non-seasonal variability"?
clim_only = False # if True, step a) only (see header); if False, both step a) and b)

#----
# define distance thresholds
#----
dist_threshold     = 1000 # find all points within this distance of the storm center (in km)
dist_threshold_deg = 25   # search radius within which to check whether JRA points fulfill dist_threshold criterion (decreases run time of script)

# loop over variables
for vv in range(0,len(vari_list)):
    if vari_list[vv] in ['diat_specific_growth_rate_surf']:
        vari = 'mu_diat'
    elif vari_list[vv] in ['sp_specific_growth_rate_surf']:
        vari = 'mu_sp'
    elif vari_list[vv] in ['totChl_emulator']:
        vari = 'totChl'
    elif vari_list[vv] in ['totChl_hr']:
        vari = 'totChl'
    elif vari_list[vv] in ['MLD_hr']:
        vari = 'HMXL_2'
    else:
        vari = vari_list[vv] 
    print(vari_list[vv],vari,'...')
    
    if vari in ['totChl']:
        if vari_list[vv] in ['totChl_hr']:
            path_string = 'CESM_HIGH_RES_chl_regridded'
            long_name   = 'total chlorophyll'
            unit        = 'mg chl m-3'
        elif vari_list[vv] in ['totChl_emulator']:
            path_string = 'CESM_totChl_emulator_regridded'
            long_name   = 'total chlorophyll'
            unit        = 'mg chl m-3'
        else:
            path_string = 'CESM_chl'
            long_name   = 'total chlorophyll'
            unit        = 'mg chl m-3'
    elif vari in ['FG_CO2_2']:
        path_string = 'CESM_fco2'
        long_name   = 'air-sea CO2 flux'
        unit        = 'mmol m-3 cm s-1'
    elif vari in ['photoC_total_surf']:
        path_string = 'CESM_total_srf_photoC_regridded'
        long_name   = 'total surface NPP'
        unit        = 'mmol m-3 cm s-1'
    elif vari in ['SST']:
        path_string = 'CESM_SST_regridded'
        long_name   = 'Sea surface temperature (potential)'
        unit        = 'deg C'
    elif vari in ['MLD']:
        path_string = 'CESM_MLD_regridded'
        long_name   = 'mixed layer depth (density-based)'
        unit        = 'm'
    elif vari in ['MLD_hr','HMXL_2']:
        path_string = 'CESM_HIGH_RES_MLD_regridded'
        long_name   = 'mixed layer depth (density-based)'
        unit        = 'm'
    elif vari in ['wind_speed']:
        path_string = 'JRA_wind_speed'
        long_name   = '10m wind speed'
        unit        = 'm s-1'
    elif vari in ['diat_Fe_lim_surf']:
        path_string = 'CESM_diat_Fe_lim_surf_regridded'
        long_name   = 'diatom surface iron limitation'
        unit        = 'n.d.'
    elif vari in ['diat_N_lim_surf']:
        path_string = 'CESM_diat_N_lim_surf_regridded'
        long_name   = 'diatom surface nitrate limitation'
        unit        = 'n.d.'
    elif vari in ['diat_P_lim_surf']:
        path_string = 'CESM_diat_P_lim_surf_regridded'
        long_name   = 'diatom surface phosphate limitation'
        unit        = 'n.d.'
    elif vari in ['diat_SiO3_lim_surf']:
        path_string = 'CESM_diat_SiO3_lim_surf_regridded'
        long_name   = 'diatom surface silicate limitation'
        unit        = 'n.d.'
    elif vari in ['sp_Fe_lim_surf']:
        path_string = 'CESM_sp_Fe_lim_surf_regridded'
        long_name   = 'SP surface iron limitation'
        unit        = 'n.d.'
    elif vari in ['sp_N_lim_surf']:
        path_string = 'CESM_sp_N_lim_surf_regridded'
        long_name   = 'SP surface nitrate limitation'
        unit        = 'n.d.'
    elif vari in ['sp_P_lim_surf']:
        path_string = 'CESM_sp_P_lim_surf_regridded'
        long_name   = 'SP surface phosphate limitation'
        unit        = 'n.d.'
    elif vari in ['diatChl_SURF']:
        path_string = 'CESM_diatChl_SURF_regridded'
        long_name   = 'diatom surface chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['spChl_SURF']:
        path_string = 'CESM_spChl_SURF_regridded'
        long_name   = 'SP surface chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['diat_light_lim_surf']:
        path_string = 'CESM_diat_light_lim_surf_regridded'
        long_name   = 'diatom surface light limitation'
        unit        = 'n.d.'
    elif vari in ['sp_light_lim_surf']:
        path_string = 'CESM_sp_light_lim_surf_regridded'
        long_name   = 'SP surface light limitation'
        unit        = 'n.d.'
    elif vari in ['diat_specific_growth_rate_surf','mu_diat']:
        path_string = 'CESM_diat_specific_growth_rate_regridded'
        long_name   = 'diatom surface specific growth rate'
        unit        = 'd-1'
    elif vari in ['sp_specific_growth_rate_surf','mu_sp']:
        path_string = 'CESM_sp_specific_growth_rate_regridded'
        long_name   = 'SP surface specific growth rate'
        unit        = 'd-1'
    elif vari in ['cloudfrac_isccp']:
        path_string = 'CESM_cloudfrac_isccp_regridded'
        long_name   = 'ISCCP cloud fraction'
        unit        = 'n.d.'
    elif vari in ['PAR_incoming']:
        path_string = 'JRA_PAR_incoming'
        long_name   = 'Incoming PAR (45% of incoming shortwave radiation)'
        unit        = 'W m-2'
    elif vari in ['photoC_zint']:
        path_string = 'CESM_total_int_photoC_regridded'
        long_name   = 'total vertically integrated NPP absolute field'
        unit        = 'mmol m-3 cm s-1 m2'
    elif vari in ['slp']:
        path_string = 'JRA_slp'
        long_name   = 'sea level pressure'
        unit        = 'Pa'        
        
    # path to data
    path1 = '/global/cfs/cdirs/m4003/cnissen/'+path_string+'/'

    #---
    # load daily clim 
    #---
    if vari_list[vv] in ['diat_specific_growth_rate_surf','sp_specific_growth_rate_surf','totChl_hr','MLD_hr']:
        file_clim = 'Climatology_'+vari_list[vv]+'_JRA_grid_1997_2018.nc' 
    #elif vari_list[vv] in ['totChl_emulator']: # if gap-filled climatology, use this
    #    file_clim = 'Climatology_'+vari_list[vv]+'_JRA_grid_1997_2018_interpolated.nc' 
    else:
        file_clim = 'Climatology_'+vari+'_JRA_grid_1997_2018.nc' 
    if vari_list[vv] in ['totChl_emulator']:
        ff2  = xr.open_dataset('/global/cfs/cdirs/m4003/cnissen/CESM_chl/'+file_clim) # load full-field climatology
        #ff2  = xr.open_dataset('/global/cfs/cdirs/m4003/cnissen/CESM_totChl_emulator_regridded/'+file_clim) # load gap-filled climatology
    else:
        ff2  = xr.open_dataset(path1+file_clim)
    if vari_list[vv] in ['totChl_emulator']: # if gap filled, use this
        data_clim = ff2['totChl_masked']#.values 
    elif vari_list[vv]=='MLD_hr':
        data_clim = ff2['MLD_hr']
    else:
        data_clim = ff2[vari]#.values 
    ff2.close()
    data_clim2 = np.concatenate((data_clim,data_clim))

    # where to save anomaly files?
    if vari_list[vv] in ['totChl_emulator']:
        savepath     = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_emulator_anomalies/'
    elif vari_list[vv] in ['totChl_hr']:
        savepath     = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_hr_anomalies/'
    elif vari_list[vv]=='MLD_hr':
        savepath     = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/MLD_hr_anomalies/'
    else:
        savepath     = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_anomalies/'
    # check existence of paths
    if not os.path.exists(savepath):
        print ('Created '+savepath)
        os.makedirs(savepath)
    
    for yy in range(1,len(year_list)):   # START AT INDEX "1" HERE 
                                         # (I always load "yy-1" as well which won't work if I start with yy=0 here!)

        print('Load year',year_list[yy])
        if vari_list[vv] in ['diat_specific_growth_rate_surf','sp_specific_growth_rate_surf','totChl_hr','MLD_hr']:
            file1 = vari_list[vv]+'_JRA_grid_'+str(year_list[yy-1])+'-01-01.nc'
            file2 = vari_list[vv]+'_JRA_grid_'+str(year_list[yy])+'-01-01.nc'
        elif vari_list[vv] in ['totChl_emulator']:
            file1 = vari+'_emulator_JRA_grid_'+str(year_list[yy-1])+'-01-01.nc'
            file2 = vari+'_emulator_JRA_grid_'+str(year_list[yy])+'-01-01.nc'
        else:
            file1 = vari+'_JRA_grid_'+str(year_list[yy-1])+'-01-01.nc'
            file2 = vari+'_JRA_grid_'+str(year_list[yy])+'-01-01.nc'
        
        # previous year
        ff2  = xr.open_dataset(path1+file1)
        if vari_list[vv] in ['totChl_emulator']:
            chl1 = ff2['totChl_masked']#.values 
        else:
            chl1 = ff2[vari]#.values 
        time1= ff2['time'].values 
        ff2.close()
        # current year
        ff2  = xr.open_dataset(path1+file2)
        if vari_list[vv] in ['totChl_emulator']:
            chl2 = ff2['totChl_masked']#.values 
        else:
            chl2 = ff2[vari]#.values 
        time2= ff2['time'].values 
        ff2.close()
        # merge the two years
        chl_all  = np.concatenate((chl1,chl2))
        time_all = np.concatenate((time1,time2))
        
        # create land-sea mask
        land_sea_mask = np.ones_like(chl_all[0,:,:])
        land_sea_mask[np.isnan(chl_all[0,:,:])] = 0

        #----
        # load sea-ice data
        path2 = '/global/cfs/cdirs/m4003/cnissen/CESM_sic/'
        file1 = 'ECOSYS_IFRAC_2_JRA_grid_'+str(year_list[yy])+'-01-01.nc'
        ff  = xr.open_dataset(path2+file1)
        data_ice = ff['ECOSYS_IFRAC_2'].values 
        print('min/max ice cover:',np.nanmin(data_ice),np.nanmax(data_ice))
        ff.close()
        # # mask chl at "high" sea-ice cover further down
        # ice_threshold = 0.5
        #----

        #---
        # count storms on each day
        #---
        # NOTE: the summed number resulting here is NOT the same as count_storms above!
        #  count_storms is for all years loaded in
        #  additionally, here, a storm is counted multiple times if it exists over multiple days
        #
        # the number resulting here (np.sum(num_storms_each_day)) is the size of the array to be defined for netcdf file

        print('Get number of storms for current year...')
        num_storms_each_day = np.zeros(365)
        for t1 in range(0,365): # loop over days in year

            tt = t1+365 # I loaded two years of data -> always process the 2nd one (to always be able to go back by 10 days!)

            aux_year  = time_all[tt].year
            aux_month = time_all[tt].month
            aux_day   = time_all[tt].day
            aux_hour  = 12 # always at noon (center of daily averages)
            ind_find_storm = np.where((np.asarray(year_storm)==aux_year) &\
                                          (np.asarray(month_storm)==aux_month) &\
                                          (np.asarray(day_storm)==aux_day) &\
                                          (np.asarray(hour_storm)==aux_hour))[0]
            num_storms_each_day[t1] = len(ind_find_storm)
            del aux_year,aux_month,aux_day,aux_hour

        print('min/max/median/mean storms per day:',np.min(num_storms_each_day),np.max(num_storms_each_day),\
                  np.median(num_storms_each_day),np.mean(num_storms_each_day))
        print('Sum of storms for current year:',np.sum(num_storms_each_day))

        #---
        # create netcdf file
        #---
        
        which_clim = '_full_field_clim' #'_gap_filled_clim' # needs to be changed for gap-filled climatology for emulator!
        if save_netcdf:
            fv = -999
            if clim_only: 
                clim_string = '_clim_only'
            else:
                clim_string = '_clim_first'
                
            if vari_list[vv] in ['totChl_emulator']:
                netcdf_name = 'Anomalies_within_'+str(dist_threshold)+'km_of_storm_center_'+\
                                    vari+'_emulator_JRA_grid_'+str(year_list[yy])+'-01-01_'+storm_string+'_subtract'+clim_string+which_clim+'.nc'
            elif vari_list[vv] in ['totChl_hr']:
                netcdf_name = 'Anomalies_within_'+str(dist_threshold)+'km_of_storm_center_'+\
                                    vari+'_hr_JRA_grid_'+str(year_list[yy])+'-01-01_'+storm_string+'_subtract'+clim_string+'.nc'
            else:
                netcdf_name = 'Anomalies_within_'+str(dist_threshold)+'km_of_storm_center_'+\
                                    vari+'_JRA_grid_'+str(year_list[yy])+'-01-01_'+storm_string+'_subtract'+clim_string+'.nc'
            if not os.path.exists(savepath+netcdf_name):
                print('Create file '+savepath+netcdf_name)
                w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                w_nc_fid.contact     = 'Cara Nissen, cara.nissen@colorado.edu'
                w_nc_fid.source_data = path1+file2
                w_nc_fid.storm_file  = pathTE+fileTE
                w_nc_fid.script      = '/global/homes/c/cnissen/scripts/save_CESM_daily_chl_anomalies_v12_with_HR_and_emulator.ipynb'
                # create dimension & variable
                w_nc_fid.createDimension('lon', len(lon)) 
                w_nc_fid.createDimension('lat', len(lat)) 
                w_nc_fid.createDimension('count_anom', np.sum(num_storms_each_day)) 
                #w_nc_fid.createDimension('time', time_all[365:,:,:].shape[0]) 

                w_nc_var1 = w_nc_fid.createVariable(vari+'_storm', 'f4',('count_anom','lat','lon'),fill_value=fv)
                w_nc_var1.long_name = long_name+' absolute field'
                w_nc_var1.units = unit
                w_nc_var1 = w_nc_fid.createVariable(vari+'_storm_anomaly4', 'f4',('count_anom','lat','lon'),fill_value=fv)
                w_nc_var1.long_name = long_name+' anomaly (subtracted daily climatology at each location)'
                w_nc_var1.units = unit
                w_nc_var1 = w_nc_fid.createVariable('sea_ice', 'f4',('count_anom','lat','lon'),fill_value=fv)
                w_nc_var1.long_name = 'sea ice concentration'
                w_nc_var1.units = 'n.d.'
                w_nc_var1 = w_nc_fid.createVariable('lat', 'f4',('lat'),fill_value=fv)
                w_nc_var1.description = 'Latitude'
                w_nc_var1.units = 'deg N'
                w_nc_var1 = w_nc_fid.createVariable('lon', 'f4',('lon'),fill_value=fv)
                w_nc_var1.description = 'Longitude (-180:180)'
                w_nc_var1.units = 'deg E'
                w_nc_var1 = w_nc_fid.createVariable('land_sea_mask', 'f4',('lat','lon'),fill_value=fv)
                w_nc_var1.long_name = 'Land-sea mask'
                #----
                # store some extra info about current storm
                w_nc_var1 = w_nc_fid.createVariable('index_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'index of storm (to know which storm imprints belong to the same storm)'
                w_nc_var1 = w_nc_fid.createVariable('max_wind_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'max. wind speed in m s-1 at current time step'
                w_nc_var1 = w_nc_fid.createVariable('min_slp_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'min. SLP speed in Pa at current time step'
                w_nc_var1 = w_nc_fid.createVariable('lon_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'Longitude of storm center'
                w_nc_var1 = w_nc_fid.createVariable('lat_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'Latitude of storm center'
                w_nc_var1 = w_nc_fid.createVariable('year_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'Year of storm center'
                w_nc_var1 = w_nc_fid.createVariable('month_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'Month of storm center'
                w_nc_var1 = w_nc_fid.createVariable('day_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'Day of storm center'
                w_nc_var1 = w_nc_fid.createVariable('hour_storm','f4',('count_anom'),fill_value=fv)
                w_nc_var1.description = 'Hour of storm center'
                #----
                
                #w_nc_var1 = w_nc_fid.createVariable('time', 'f4',('time'),fill_value=fv)
                #w_nc_var1.units = 'days since '+str(year_list2[0])+'-01-01 00:00:00'
                #w_nc_var1.calendar = 'noleap'   

                # write lat/lon to file
                w_nc_fid.variables['land_sea_mask'][:] = land_sea_mask
                w_nc_fid.variables['lat'][:] = lat
                w_nc_fid.variables['lon'][:] = lon

                w_nc_fid.close()

        #----
        # get all points within 1000km, create a mask to be used for averaging, get avg CHL within that mask for the 10 days preceding the storm day -> calculate anomaly
        #----
        # see: https://gis.stackexchange.com/questions/450878/finding-all-latitude-and-longitudes-within-x-km-range-from-input-location-using

        storm_count = 0 # used for indexing when writing netcdf
        for t1 in tqdm(range(0,365)): # loop over days in year

            tt = t1+365 # I loaded two years of data -> always process the 2nd one (to always be able to go back by 10 days!)

            aux_year  = time_all[tt].year
            aux_month = time_all[tt].month
            aux_day   = time_all[tt].day
            aux_hour  = 12 # always at noon (center of daily averages)

            ind_find_storm = np.where((np.asarray(year_storm)==aux_year) &\
                                      (np.asarray(month_storm)==aux_month) &\
                                      (np.asarray(day_storm)==aux_day) &\
                                      (np.asarray(hour_storm)==aux_hour))[0]
        #    print('Number of storms:',len(ind_find_storm))
            del aux_year,aux_month,aux_day,aux_hour

            if len(ind_find_storm)>0: # if any storm was found, continue

                # extract all storm positions
                aux_index    = np.asarray(index_storm)[ind_find_storm]
                aux_min_slp  = np.asarray(slp_center)[ind_find_storm]
                aux_max_wind = np.asarray(wind_max)[ind_find_storm]
                aux_lon = np.asarray(lon_storm)[ind_find_storm]
                aux_lat = np.asarray(lat_storm)[ind_find_storm]
                aux_year  = np.asarray(year_storm)[ind_find_storm]
                aux_month = np.asarray(month_storm)[ind_find_storm]
                aux_day   = np.asarray(day_storm)[ind_find_storm]
                aux_hour   = np.asarray(hour_storm)[ind_find_storm]

                data_anom_all_storms = np.zeros_like(chl_all[0,:,:])
                mask_all_storms      = np.zeros_like(chl_all[0,:,:])
                for ss in range(0,len(ind_find_storm)): # loop over storms on current day

                    lon_point, lat_point = aux_lon[ss],aux_lat[ss] # current storm position
                    lon2,lat2 = np.meshgrid(lon,lat) # JRA mesh
                    # reduce to array to within +-XX° of current storm location of interest (for 1000km, this should be enough)
                    dist_threshold_deg2 = 5
                    ind_lat = np.where((lat>(lat_point-dist_threshold_deg2)) & (lat<(lat_point+dist_threshold_deg2)))[0]
                    ind_lon = np.where((lon>(lon_point-dist_threshold_deg2)) & (lon<(lon_point+dist_threshold_deg2)))[0]
                    lat2 = lat2[ind_lat,:][:,ind_lon]
                    lon2 = lon2[ind_lat,:][:,ind_lon]
                    lon2,lat2 = lon2.ravel(),lat2.ravel()
                    # find closest node in reduced JRA mesh
                    lat2_rad = [radians(x) for x in lat2]
                    lon2_rad = [radians(x) for x in lon2]
                    index_closest_node, distance_closest_node = get_closest_grid_point(lon_point, lat_point,\
                                                                                       lon2_rad,lat2_rad)
                    # get corresponding indices in full JRA mesh
                    aux1 = lon2[index_closest_node]
                    aux2 = lat2[index_closest_node]
                    mm1 = np.where(lat==aux2)[0]
                    nn1 = np.where(lon==aux1)[0]
                    del aux1,aux2,ind_lat,ind_lon,lat2,lon2,index_closest_node,distance_closest_node

                    # get mask on JRA mesh for all points within XXkm of storm position
                #    print('lon/lat of storm at current time:',lon[nn1],lat[mm1])
                    lon_point,lat_point = lon[nn1],lat[mm1]
                    input_point = (lat_point, lon_point)

                    # input points: reorganize grid cells of JRA mesh: 
                    lon2,lat2 = np.meshgrid(lon,lat)
                    # reduce to array to within +-XX° of current storm location of interest (for 1000km, this should be enough)
                    # lon runs from -180:180, i.e., there is a jump from -180 to +180 at the dateline in the Pacific!
                    # for all lon_point close enough to the dateline (within dist_threshold_deg), the search for 
                    #      indices needs to be adapted!
                    if lon_point<(-180+dist_threshold_deg): # storm is to the right of the dateline (within dist_threshold_deg)
                        ind_lon = np.where(((lon>(lon_point-dist_threshold_deg)) & (lon<(lon_point+dist_threshold_deg))) | (lon>(180-dist_threshold_deg-(180+lon_point))))[0]
                    elif lon_point>(180-dist_threshold_deg): # storm is to the left of the dateline (within dist_threshold_deg)
                        ind_lon = np.where(((lon>(lon_point-dist_threshold_deg)) & (lon<(lon_point+dist_threshold_deg))) | (lon<(-180+dist_threshold_deg-(180-lon_point))))[0]
                    else: # distance to dateline is >dist_threshold_deg
                        ind_lon = np.where((lon>(lon_point-dist_threshold_deg)) & (lon<(lon_point+dist_threshold_deg)))[0]
                    ind_lat = np.where((lat>(lat_point-dist_threshold_deg)) & (lat<(lat_point+dist_threshold_deg)))[0]    
                    lat2 = lat2[ind_lat,:][:,ind_lon]
                    lon2 = lon2[ind_lat,:][:,ind_lon]
                    #print(lat2.shape)
                    points_jra = []
                    for mm in range(0,lat2.shape[0]):
                        for nn in range(0,lon2.shape[1]):
                            aux = (lat2[mm,nn],lon2[mm,nn])
                            points_jra.append(aux)
                            del aux

                    # loop over all JRA points in the area and get all points that fulfill distance criterion
                    points_filtered = []
                    for point in points_jra:
                        if distance(input_point, point).km < dist_threshold:
                            points_filtered.append(point)

                    del points_jra,lat2,lon2,input_point

                    lon2,lat2 = np.meshgrid(lon,lat) # re-define to get the size of the full JRA mesh
                    lon2[lon2<0] = lon2[lon2<0]+360
                    chl_mask  = np.zeros_like(lat2) # initialize data_mask (to be used in averaging)
                    chl_mask  = mask_chl_field(points_filtered,lat,lon,chl_mask)
                    chl_mask    = np.ma.masked_where(np.isnan(chl_all[0,:,:]),chl_mask) # apply land-sea mask

                    if test_plot_mask:
                        fig  = plt.figure(figsize=(5,2.5))
                        plt.pcolor(lon2,lat2,chl_mask,shading='auto')
                        # plt.plot(nn1,mm1,'wo')
                        if lon[nn1]<0:
                            lon_plot = lon[nn1]+360
                        else:
                            lon_plot = lon[nn1]
                        plt.plot(lon_plot,lat[mm1],'bx')
                        plt.show()
                        del lon_plot

                    #----
                    # get chl avg within "storm-impacted region" for the 10 preceeding days
                    #----           
                    aux_chl = chl_all[tt-10:tt,:,:] # reduce to 10 preceeding days

                    area_masked = np.copy(area)
                    area_masked = np.ma.masked_where(np.isnan(chl_all[0,:,:]),area_masked) # apply land-sea mask
                    area_masked = np.tile(area_masked,[10,1,1]) # bring to the same size as aux_chl

                    aux_chl = np.multiply(aux_chl,chl_mask) # 10 x lat x lon
                    #print('aux_chl.shape',aux_chl.shape)
                 #   aux_chl_avg_10 = np.nanmean(aux_chl,axis=0)
                 #   aux_chl_avg_7  = np.nanmean(aux_chl[3:,:,:],axis=0)
                 #   aux_chl_avg_5  = np.nanmean(aux_chl[5:,:,:],axis=0)

                    if test_plot_masked_field:
                        print('aux_index[ss]:',aux_index[ss])
                        # calculate anomaly in "storm-impacted region"
                        chl_current = np.copy(chl_all)[tt,:,:]
                        
                        indSO = np.where(lat<-30)[0]
                        
                        fig  = plt.figure(figsize=(8,2.5))
                        plt.pcolor(lon2[indSO,:],lat2[indSO,:],chl_current[indSO,:],cmap=plt.cm.RdYlBu_r,vmin=0,vmax=100,shading='auto')
                        plt.colorbar()
                        plt.show()
                        
                        #chl_current[chl_mask==1] = chl_current[chl_mask==1]-chl_avg_5 # within 1000km of storm center, subtract mean field of the preceeding 10 days
                        chl_current[chl_mask==1] = chl_current[chl_mask==1]
                        chl_current[chl_mask==0] = 0 # outside of storm-impacted area, set field to zero
                        if vari in ['totChl']:
                            cmap1 = plt.cm.viridis
                        elif vari in ['FG_CO2_2']:
                            cmap1 = plt.cm.RdBu_r
                            max1  = np.max(np.abs(aux_chl[0,:,:]))
                        else:
                            cmap1 = plt.cm.RdYlBu_r
                        fig  = plt.figure(figsize=(8,2.5))
                        if vari in ['totChl']:
                            plt.pcolor(lon2[indSO,:],lat2[indSO,:],chl_current[indSO,:],cmap=cmap1,shading='auto')
                        elif vari in ['FG_CO2_2']:
                            plt.pcolor(lon2[indSO,:],lat2[indSO,:],chl_current[indSO,:],cmap=cmap1,vmin=-0.012,vmax=0.012,shading='auto') #vmin=-1*max1,vmax=max1
                        else:
                            plt.pcolor(lon2[indSO,:],lat2[indSO,:],chl_current[indSO,:],cmap=cmap1,vmin=0,vmax=100,shading='auto')
                        plt.colorbar()
                        if lon[nn1]<0:
                            lon_plot = lon[nn1]+360
                        else:
                            lon_plot = lon[nn1]
                        plt.plot(lon_plot,lat[mm1],'bx')
                        plt.show()
                    #    del lon_plot,chl_current,indSO

                    if save_netcdf:
                       # absolute field: 
                        chl_current_abs  = np.copy(chl_all)[tt,:,:]
                        chl_current_abs[chl_mask==1] = chl_current_abs[chl_mask==1]
                        chl_current_abs[chl_mask==0] = fv # outside of storm-impacted area, set field to fill value
                        chl_current_abs[chl_mask.mask==True] = fv
                        #-----
                        # anomaly 4: CORRECTION -> subtract clim, then calculate avg for 5 days before from this anomaly (not the original abs. field)!
                        chl_current4 = np.copy(chl_all)[tt,:,:]
                        chl_current4[chl_mask==1] = chl_current4[chl_mask==1]-(data_clim[tt-365,:,:].values)[chl_mask==1] # subtract daily clim.
                        if not clim_only:
                            #-------
                            # NEW VERSION: 
                            #  always subtract the same 5 days, no matter if +1, +2 etc is chosen!
                            aux_chl2 = chl_all[tt-5-days_shift:tt-days_shift,:,:] # reduce to 5 preceeding days
                            aux_chl2 = np.multiply(aux_chl2,chl_mask) # 5 x lat x lon
                            for dd in range(0,5): # get anomaly for each of the days preceding the storm
                                aux_chl2[dd,:,:] = aux_chl2[dd,:,:]-data_clim2[tt-5-days_shift:tt-days_shift,:,:][dd,:,:] # subtract daily clim.#-data_clim[tt-365,:,:]
                            aux_chl_avg2 = np.nanmean(aux_chl2,axis=0)
                            # subtract this from the anomaly
                            chl_current4[chl_mask==1] = chl_current4[chl_mask==1]-aux_chl_avg2[chl_mask==1] # within 1000km of storm center, subtract climatology at each location, then subtract avg of 10 prec. days at each location
                            #-------
                           # #---
                           # # COMMENT FROM HERE TO "END" for subtract_clim_only
                           # # get avg for the 5 days preceding the storm
                           # aux_chl2 = chl_all[tt-5:tt,:,:] # reduce to 5 preceeding days
                           # aux_chl2 = np.multiply(aux_chl2,chl_mask) # 5 x lat x lon
                           # for dd in range(0,5): # get anomaly for each of the days preceding the storm
                           #     aux_chl2[dd,:,:] = aux_chl2[dd,:,:]-data_clim2[tt-5:tt,:,:][dd,:,:] # subtract daily clim.#-data_clim[tt-365,:,:]
                           # aux_chl_avg2 = np.nanmean(aux_chl2,axis=0)
                           # # subtract this from the anomaly
                           # chl_current4[chl_mask==1] = chl_current4[chl_mask==1]-aux_chl_avg2[chl_mask==1] # within 1000km of storm center, subtract climatology at each location, then subtract avg of 10 prec. days at each location
                           # # END
                           # #------
                        chl_current4[chl_mask==0] = fv # outside of storm-impacted area, set field to fill value
                        chl_current4[chl_mask.mask==True] = fv
                      #  del aux_chl2
                        #------
                        
                        # sea ice
                        ice_aux = np.copy(data_ice)[tt-365,:,:] # only loaded current year
                        ice_aux[chl_mask==0] = fv # outside of storm-impacted area, set field to fill value

                        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                        w_nc_fid.variables[vari+'_storm'][storm_count,:,:]          = chl_current_abs
                        w_nc_fid.variables[vari+'_storm_anomaly4'][storm_count,:,:] = chl_current4 # subtract avg of 5 prec. days at each location
                        w_nc_fid.variables['sea_ice'][storm_count]     = ice_aux
                        w_nc_fid.variables['index_storm'][storm_count] = aux_index[ss]
                        w_nc_fid.variables['max_wind_storm'][storm_count] = aux_max_wind[ss]
                        w_nc_fid.variables['min_slp_storm'][storm_count]  = aux_min_slp[ss]
                        w_nc_fid.variables['lon_storm'][storm_count]   = lon_point
                        w_nc_fid.variables['lat_storm'][storm_count]   = lat_point
                        w_nc_fid.variables['year_storm'][storm_count]  = aux_year[ss]
                        w_nc_fid.variables['month_storm'][storm_count] = aux_month[ss]
                        w_nc_fid.variables['day_storm'][storm_count]   = aux_day[ss]
                        w_nc_fid.variables['hour_storm'][storm_count]  = aux_hour[ss]
                        #---
                        # also store wind speed and SLP!
                        #---
                        w_nc_fid.close()  

                        del chl_current4,ice_aux
                        # del chl_current2,chl_current6,chl_current7,chl_current_abs
                
                        #print(storm_count,np.nanmax(chl_current4))

                    # update storm_count
                    storm_count = storm_count+1

                    # combine all storms of a single day into one array (to think about: could there ever be overlap??? If yes, that's a problem in this step)
                    mask_all_storms[chl_mask==1] = 1

                    del ind_lat,ind_lon,lon2,lat2,points_filtered,aux_chl,chl_mask
                del aux_lon,aux_lat,aux_year,aux_month,aux_day,aux_hour

                # plot the resulting storm mask (all storms on current day)
                if test_plot_combined_mask:
                    lon2,lat2 = np.meshgrid(lon,lat) # re-define to get the size of the full JRA mesh
                    lon2[lon2<0] = lon2[lon2<0]+360
                    indSO = np.where(lat<-30)[0]
                    fig  = plt.figure(figsize=(7,2.5))
                    plt.pcolor(lon2[indSO,:],lat2[indSO,:],mask_all_storms[indSO,:],shading='auto')
                    plt.show()
                

totChl_emulator totChl ...
Load year 2013
min/max ice cover: 1.2223171e-16 1.0
Get number of storms for current year...
min/max/median/mean storms per day: 2.0 11.0 6.0 5.789041095890411
Sum of storms for current year: 2113.0
Create file /global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/totChl_emulator_anomalies/Anomalies_within_1000km_of_storm_center_totChl_emulator_JRA_grid_2013-01-01_all_at_noon_plus_4_days_subtract_clim_first_gap_filled_clim.nc


  0%|          | 0/365 [00:00<?, ?it/s]/global/homes/c/cnissen/.conda/envs/myenv/lib/python3.9/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'lat2' of function 'get_closest_grid_point'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../tmp/ipykernel_558905/3917996291.py", line 5:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/global/homes/c/cnissen/.conda/envs/myenv/lib/python3.9/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'lon2' of function 'get_closest_grid_point'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecati

Load year 2014
min/max ice cover: 8.2041245e-16 1.0
Get number of storms for current year...
min/max/median/mean storms per day: 1.0 10.0 5.0 5.5917808219178085
Sum of storms for current year: 2041.0
Create file /global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/totChl_emulator_anomalies/Anomalies_within_1000km_of_storm_center_totChl_emulator_JRA_grid_2014-01-01_all_at_noon_plus_4_days_subtract_clim_first_gap_filled_clim.nc


100%|██████████| 365/365 [1:12:48<00:00, 11.97s/it]


Load year 2015
min/max ice cover: 1.1540176e-15 1.0
Get number of storms for current year...
min/max/median/mean storms per day: 1.0 11.0 5.0 5.517808219178082
Sum of storms for current year: 2014.0
Create file /global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/totChl_emulator_anomalies/Anomalies_within_1000km_of_storm_center_totChl_emulator_JRA_grid_2015-01-01_all_at_noon_plus_4_days_subtract_clim_first_gap_filled_clim.nc


...

Load year 2017
min/max ice cover: 7.819186e-16 1.0
Get number of storms for current year...
min/max/median/mean storms per day: 2.0 12.0 6.0 5.556164383561644
Sum of storms for current year: 2028.0
Create file /global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/totChl_emulator_anomalies/Anomalies_within_1000km_of_storm_center_totChl_emulator_JRA_grid_2017-01-01_all_at_noon_plus_4_days_subtract_clim_first_gap_filled_clim.nc


100%|██████████| 365/365 [1:12:08<00:00, 11.86s/it]


Load year 2018
min/max ice cover: 1.8208962e-16 1.0
Get number of storms for current year...
min/max/median/mean storms per day: 0.0 11.0 6.0 5.8054794520547945
Sum of storms for current year: 2119.0
Create file /global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/totChl_emulator_anomalies/Anomalies_within_1000km_of_storm_center_totChl_emulator_JRA_grid_2018-01-01_all_at_noon_plus_4_days_subtract_clim_first_gap_filled_clim.nc


100%|██████████| 365/365 [1:14:15<00:00, 12.21s/it]

CPU times: user 5h 1min 49s, sys: 2h 12min 31s, total: 7h 14min 20s
Wall time: 7h 19min 44s
